This uses the Jupyter Racket kernel from https://github.com/rmculpepper/iracket.

In [1]:
#lang iracket/lang #:require sicp

## Q1.1

In [2]:
10
;; 10

10

In [3]:
(+ 5 3 4)
;; 12

12

In [4]:
(- 9 1)
;; 8

8

In [5]:
(/ 6 2)
;; 3

3

In [6]:
(+ (* 2 4) (- 4 6))
;; 6

6

In [7]:
(define a 3)
(define b (+ a 1))
(+ a b (* a b))
;; 19

19

In [8]:
(= a b)
;; #f

#f

In [9]:
(if (and (> b a) (< b (* a b)))
    b
    a)
;; 4

4

In [10]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))
;; 16

16

In [11]:
(+ 2 (if (> b a) b a))
;; 6

6

In [12]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))
;; 16

16

## Q1.2

In [13]:
(/ (+ 5 4 (- 2 (- 3 (+ 6 (/ 4 5)))))
   (* 3 (- 6 2) (- 2 7)))

-37/150

## Q1.3

In [14]:
(define (sum s) (apply + s))
(define (square x) (* x x))
(define (sum-squared-largest a b c)
    (define minval (min a b c))
    (define vals (list a b c))
    (- (sum (map square vals)) (* minval minval)))

(sum-squared-largest 2 3 4)

25

## Q1.4

```python
# return a + |b|
def a_plus_abs_b(a, b):
    if b > 0:
        return a + b
    else:
        return a - b
```

## Q1.5

Applicative-order evaluation doesn't terminate. When we expand, we get
```scheme
(test 0 (p))
(test 0 (p))
...
```
since the interpreter keeps trying to reduce the recursive operand `(p)` first.


Normal-order evaluation returns 0 since it short-circuits the recursive evaluation by reducing the operator first.
```scheme
(test 0 (p))
(if (= 0 0) 0 (p))
(if #t 0 (p))
0
```

## Q1.6

Under the normal special form `if` statement, the predicate gets evaluated before the other arguments despite using applicative-order evaluation.
```scheme
;; Normal if
(sqrt-iter 1 2)
(if (good-enough? 1 2)
    1
    (sqrt-iter (improve 1 2) 2)))
(if #f
    1
    (sqrt-iter (improve 1 2) 2)))
(sqrt-iter (improve 1 2) 2)
(sqrt-iter 1.5 2)
...
```

New-if is instead treated as an ordinary procedure, so the `(sqrt-iter (improve guess x) x)` alternative gets infinitely expanded.
```scheme
;; New if
(sqrt-iter 1 2)
(new-if (good-enough? 1 2)
    1
    (sqrt-iter (improve 1 2) 2)))
(new-if (good-enough? 1 2)
    1
    (sqrt-iter 1.5 2)))
(new-if (good-enough? 1 2)
    1
    (new-if (good-enough? 1.5 2)
        1.5
        (sqrt-iter (improve 1.5 2) 2)))))
...
```

## Q1.7

`sqrt(0.0001)` is evaluated as 0.0323, significantly off from the its true value of 0.01. Here the tolerance of 0.001 is too high.\
`sqrt(1e13)` does not terminate because the improve step runs out of precision to improve while the difference remains above 0.001.

In [15]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (average guess (/ x guess)))
(define (good-enough? guess x) (< (abs (- (square guess) x)) 0.001))
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))
(define (sqrt x) (sqrt-iter 1.0 x))
(display (sqrt 0.0001)) (newline)

0.03230844833048122


In [16]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (average guess (/ x guess)))
(define (good-enough? guess x) (and
        (display guess)
        (display " ")
        (display (abs (- (square guess) x))) (newline)
        (= guess (improve guess x))))
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))
(define (sqrt x) (sqrt-iter 1.0 x))
(display (sqrt 1e13)) (newline)

1.0 9999999999999.0
5000000000000.5 2.4999999999995e+25
2500000000001.25 6.24999999999625e+24
1250000000002.625 1.5624999999965625e+24
625000000005.3125 3.906249999966406e+23
312500000010.65625 9.765624999666016e+22
156250000021.32813 2.441406249666504e+22
78125000042.66406 6.103515621666259e+21
39062500085.33203 1.525878902916565e+21
19531250170.666016 3.814697232291413e+20
9765625341.333006 9.536742830728538e+19
4882813182.666485 2.384185457682161e+19
2441407615.3330994 5960461144206451000.0
1220705855.6654043 1490112786055807000.0
610357023.8235396 372525696530728900.0
305186703.8384698 93128924199789870.0
152609735.33285388 23279731318363710.0
76337630.97598314 5817433903025381.0
38234313.983767174 1451862765809294.0
19247929.57491261 360482792920795.56
9883732.98644831 87688177747406.44
5447748.229088079 19677960767532.305
3641684.517987917 3261866128552.8887
3193833.2403111905 200570766916.67773
3162433.547242504 985940724.8066406
3162277.6640104805 24299.58203125
3162277.6601683

Stopping when the guess's change becomes relatively small helps in both cases.

In [17]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (average guess (/ x guess)))
(define (good-enough? guess x) (
        < (abs (- guess (improve guess x))) (* 0.001 guess)))
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))
(define (sqrt x) (sqrt-iter 1.0 x))
(display (sqrt 0.0001)) (newline)
(display (sqrt 1e13)) (newline)

0.010000714038711746
3162433.547242504


## Q1.8

In [18]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (/ (+ (/ x (square guess)) (* 2 guess)) 3))
(define (good-enough? guess x) (
        < (abs (- guess (improve guess x))) (* 0.001 guess)))
(define (cubrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (cubrt-iter (improve guess x) x)))
(define (cubrt x) (cubrt-iter 1.0 x))
(display (cubrt 0.001)) (newline)
(display (cubrt 1e12)) (newline)
(display (cubrt 1e13)) (newline)

0.10001409266436927
10001.829838573718
21545.34186023313
